# Visualizations

Views of data primarily taken from the shard `redux<yyyy>` files/folders.

## Bundle plotter: includes TMLD estimates


If the TMLD CSV file exists the locations of the bottom of the mixed layer appear as red dots: If that feature is selected.

In [3]:
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from datetime import datetime, timedelta

def get_input_with_default(prompt, default):
    """Get user input with default value."""
    response = input(f"{prompt} ").strip()
    return response if response else str(default)

# Sensor configuration (dissolved oxygen back to 300)
SENSORS = {
    'temperature': {'low': 7.0, 'high': 20.0, 'units': '°C'},
    'salinity': {'low': 32.0, 'high': 34.0, 'units': 'PSU'},
    'density': {'low': 1024.0, 'high': 1028.0, 'units': 'kg/m³'},
    'dissolvedoxygen': {'low': 50.0, 'high': 300.0, 'units': 'µmol/kg'}
}

print("Scanning for redux folders...")
available_years = []
for year in range(2014, 2027):
    redux_dir = Path(f"~/redux{year}").expanduser()
    if redux_dir.exists():
        profile_count = len(list(redux_dir.glob("*.nc")))
        if profile_count > 0:
            print(f"  redux{year}: {profile_count} profiles")
            response = get_input_with_default(f"    Include {year}? [y/n] (default y):", "y").lower()
            if response == 'y':
                available_years.append(year)

if not available_years:
    print("No years selected")
else:
    print(f"\nSelected years: {available_years}")
    
    # Select number of sensors
    num_sensors = int(get_input_with_default("How many sensors to plot? (1 or 2, default 1):", "1"))
    
    selected_sensors = []
    sensor_configs = []
    
    # Select sensors
    for i in range(num_sensors):
        print(f"\nSensor {i+1} - Available types:")
        for idx, sensor in enumerate(SENSORS.keys(), 1):
            print(f"  {idx}. {sensor}")
        
        sensor_choice = input(f"Select sensor {i+1} (1-4): ").strip()
        sensor_idx = int(sensor_choice) if sensor_choice else 1
        selected_sensor = list(SENSORS.keys())[sensor_idx - 1]
        
        # Get x-axis limits
        default_low = SENSORS[selected_sensor]['low']
        default_high = SENSORS[selected_sensor]['high']
        units = SENSORS[selected_sensor]['units']
        
        x_low = float(get_input_with_default(f"  Low {selected_sensor} (default {default_low}):", default_low))
        x_high = float(get_input_with_default(f"  High {selected_sensor} (default {default_high}):", default_high))
        
        selected_sensors.append(selected_sensor)
        sensor_configs.append({'name': selected_sensor, 'low': x_low, 'high': x_high, 'units': units})
        
        print(f"  Selected: {selected_sensor}, range {x_low} to {x_high} {units}")
    
    # Load profile files for each sensor
    all_profile_files = {}
    for sensor in selected_sensors:
        profile_files = []
        for year in available_years:
            redux_dir = Path(f"~/redux{year}").expanduser()
            year_files = sorted(list(redux_dir.glob(f"*_{sensor}_*.nc")))
            profile_files.extend(year_files)
        all_profile_files[sensor] = profile_files
        print(f"\n{sensor}: {len(profile_files)} profiles")
    
    # Use the sensor with most profiles for indexing
    max_profiles = max(len(files) for files in all_profile_files.values())
    
    if max_profiles == 0:
        print("No profiles found")
    else:
        def extract_profile_info(filename):
            """Extract year, day, and profile number from filename."""
            parts = filename.stem.split('_')
            year = int(parts[4])
            doy = int(parts[5])
            profile_num = int(parts[7])
            return year, doy, profile_num
        
        def check_time_gap(files, start_idx, end_idx):
            """Check if there's a >2 day gap between consecutive profiles."""
            if len(files) == 0 or start_idx >= len(files):
                return False
            for i in range(start_idx, min(end_idx - 1, len(files) - 1)):
                parts1 = files[i].stem.split('_')
                parts2 = files[i + 1].stem.split('_')
                
                year1, doy1 = int(parts1[4]), int(parts1[5])
                year2, doy2 = int(parts2[4]), int(parts2[5])
                
                date1 = datetime(year1, 1, 1) + timedelta(days=doy1 - 1)
                date2 = datetime(year2, 1, 1) + timedelta(days=doy2 - 1)
                
                if (date2 - date1).days > 2:
                    return True
            return False
        
        def plot_bundle(nProfiles, index0):
            """Plot a bundle of consecutive profiles."""
            
            if nProfiles == 0:
                plt.figure(figsize=(10, 8))
                plt.text(0.5, 0.5, 'Select nProfiles > 0', ha='center', va='center', transform=plt.gca().transAxes)
                plt.show()
                return
            
            start_idx = index0 - 1
            end_idx = min(start_idx + nProfiles, max_profiles)
            
            if start_idx >= max_profiles:
                plt.figure(figsize=(10, 8))
                plt.text(0.5, 0.5, f'Index {index0} exceeds available profiles', ha='center', va='center', transform=plt.gca().transAxes)
                plt.show()
                return
            
            # Determine plot width
            fig_width = 12 if num_sensors == 1 else 18
            fig, ax = plt.subplots(figsize=(fig_width, 8))
            
            # Calculate x-axis range and offsets
            if num_sensors == 1:
                a, b = sensor_configs[0]['low'], sensor_configs[0]['high']
                x_min, x_max = a, b
                sensor1_plot_range = (a, b)
                sensor2_plot_range = None
            else:
                a, b = sensor_configs[0]['low'], sensor_configs[0]['high']
                c, d = sensor_configs[1]['low'], sensor_configs[1]['high']
                
                # Sensor 1: a to (2b - a)
                # Sensor 2: (2c - d) to d
                sensor1_plot_range = (a, 2*b - a)
                sensor2_plot_range = (2*c - d, d)
                
                x_min = min(a, 2*c - d)
                x_max = max(2*b - a, d)
            
            # Check for time gap
            first_sensor_files = all_profile_files[selected_sensors[0]]
            has_time_gap = check_time_gap(first_sensor_files, start_idx, end_idx)
            
            # Plot each sensor
            for sensor_idx, (sensor, config) in enumerate(zip(selected_sensors, sensor_configs)):
                profile_files = all_profile_files[sensor]
                
                # Calculate offset for plotting
                if num_sensors == 2 and sensor_idx == 1:
                    # Map sensor 2 range [c, d] to plot range [2c-d, d]
                    # offset = (2c - d) - c = c - d
                    offset = config['low'] - config['high']
                else:
                    offset = 0
                
                # Plot profiles
                for i in range(start_idx, min(end_idx, len(profile_files))):
                    try:
                        ds = xr.open_dataset(profile_files[i])
                        sensor_data = ds[sensor].values
                        depth = ds['depth'].values
                        
                        valid_mask = ~(np.isnan(sensor_data) | np.isnan(depth))
                        if np.any(valid_mask):
                            data_clean = sensor_data[valid_mask] + offset
                            depth_clean = depth[valid_mask]
                            
                            color = 'blue' if sensor_idx == 0 else 'red'
                            ax.plot(data_clean, depth_clean, '-', color=color, markersize=1, alpha=0.6, linewidth=1)
                    except Exception:
                        continue
            
            # Set up axes
            ax.set_ylabel('Depth (m)', fontsize=12)
            ax.set_xlim(x_min, x_max)
            ax.set_ylim(200, 0)
            ax.grid(True, alpha=0.3)
            
            # Create custom tick marks for dual sensors
            if num_sensors == 1:
                ax.set_xlabel(f'{sensor_configs[0]["name"].capitalize()} ({sensor_configs[0]["units"]})', fontsize=12)
            else:
                # Clear default ticks
                ax.set_xlabel('')
                ax.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
                
                # Add custom tick bars below
                fig.text(0.5, 0.08, 
                        f'{sensor_configs[0]["name"].capitalize()} (blue): {sensor_configs[0]["low"]} to {sensor_configs[0]["high"]} {sensor_configs[0]["units"]}',
                        ha='center', fontsize=10, color='blue')
                fig.text(0.5, 0.04, 
                        f'{sensor_configs[1]["name"].capitalize()} (red): {sensor_configs[1]["low"]} to {sensor_configs[1]["high"]} {sensor_configs[1]["units"]}',
                        ha='center', fontsize=10, color='red')
            
            # Add Time Gap warning
            if has_time_gap:
                ax.text(0.95, 0.05, 'Time Gap', transform=ax.transAxes,
                       fontsize=20, fontweight='bold', ha='right', va='bottom',
                       bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', linewidth=2))
            
            # Title
            if len(first_sensor_files) > start_idx:
                first_year, first_doy, first_profile = extract_profile_info(first_sensor_files[start_idx])
                last_idx = min(end_idx - 1, len(first_sensor_files) - 1)
                last_year, last_doy, last_profile = extract_profile_info(first_sensor_files[last_idx])
                title = f'Bundle: {first_year}-{first_doy:03d}-{first_profile} to {last_year}-{last_doy:03d}-{last_profile}'
                ax.set_title(title, fontsize=14)
            
            plt.tight_layout()
            if num_sensors == 2:
                plt.subplots_adjust(bottom=0.12)
            plt.show()
        
        # Create interactive widgets
        nProfiles_slider = widgets.IntSlider(value=1, min=0, max=180, step=1, description='nProfiles:', continuous_update=False)
        index0_slider = widgets.IntSlider(value=1, min=1, max=max_profiles, step=1, description='index0:', continuous_update=False)
        
        # Navigation buttons
        def on_minus_minus(b):
            step = max(1, nProfiles_slider.value // 2)
            index0_slider.value = max(1, index0_slider.value - step)
        
        def on_minus(b):
            index0_slider.value = max(1, index0_slider.value - 1)
        
        def on_plus(b):
            index0_slider.value = min(max_profiles, index0_slider.value + 1)
        
        def on_plus_plus(b):
            step = max(1, nProfiles_slider.value // 2)
            index0_slider.value = min(max_profiles, index0_slider.value + step)
        
        btn_minus_minus = widgets.Button(description='--')
        btn_minus = widgets.Button(description='-')
        btn_plus = widgets.Button(description='+')
        btn_plus_plus = widgets.Button(description='++')
        
        btn_minus_minus.on_click(on_minus_minus)
        btn_minus.on_click(on_minus)
        btn_plus.on_click(on_plus)
        btn_plus_plus.on_click(on_plus_plus)
        
        nav_buttons = widgets.HBox([btn_minus_minus, btn_minus, btn_plus, btn_plus_plus])
        
        # Create interactive plot
        interactive_plot = widgets.interactive(plot_bundle, nProfiles=nProfiles_slider, index0=index0_slider)
        
        # Display widgets
        display(interactive_plot)
        display(nav_buttons)


Scanning for redux folders...
  redux2015: 2636 profiles


    Include 2015? [y/n] (default y):  n


  redux2016: 11812 profiles


    Include 2016? [y/n] (default y):  n


  redux2017: 5636 profiles


...

How many sensors to plot? (1 or 2, default 1):  1



Sensor 1 - Available types:
  1. temperature
  2. salinity
  3. density
  4. dissolvedoxygen


Select sensor 1 (1-4):  4
  Low dissolvedoxygen (default 50.0):  
  High dissolvedoxygen (default 300.0):  


  Selected: dissolvedoxygen, range 50.0 to 300.0 µmol/kg

dissolvedoxygen: 2193 profiles


interactive(children=(IntSlider(value=1, continuous_update=False, description='nProfiles:', max=180), IntSlide…

## Bundle plot animation generator

In [3]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import xarray as xr
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

def get_input_with_default(prompt, default):
    """Get user input with default value."""
    response = input(f"{prompt} ").strip()
    return response if response else default

def load_tmld_data():
    """Load TMLD data if available."""
    try:
        return pd.read_csv('tmld_estimates.csv')
    except FileNotFoundError:
        return pd.DataFrame()

def check_time_gap(files, start_idx, end_idx):
    """Check if there's a >2 day gap between consecutive profiles."""
    for i in range(start_idx, end_idx - 1):
        parts1 = files[i].stem.split('_')
        parts2 = files[i + 1].stem.split('_')
        
        year1, doy1 = int(parts1[4]), int(parts1[5])
        year2, doy2 = int(parts2[4]), int(parts2[5])
        
        date1 = datetime(year1, 1, 1) + timedelta(days=doy1 - 1)
        date2 = datetime(year2, 1, 1) + timedelta(days=doy2 - 1)
        
        time_diff = (date2 - date1).days
        
        if time_diff > 2:
            return True
    
    return False

def calculate_mean_profile(files, start_idx, end_idx):
    """Calculate mean and std profiles from bundle."""
    depth_grid = np.linspace(0, 200, 201)
    temp_profiles = []
    
    for i in range(start_idx, end_idx):
        try:
            ds = xr.open_dataset(files[i])
            temperature = ds['temperature'].values
            depth = ds['depth'].values
            
            valid_mask = ~(np.isnan(temperature) | np.isnan(depth))
            if np.any(valid_mask):
                temp_clean = temperature[valid_mask]
                depth_clean = depth[valid_mask]
                
                if len(temp_clean) > 1:
                    f = interp1d(depth_clean, temp_clean, bounds_error=False, fill_value=np.nan)
                    temp_interp = f(depth_grid)
                    # Only add if we have some valid data
                    if not np.all(np.isnan(temp_interp)):
                        temp_profiles.append(temp_interp)
        except:
            continue
    
    if len(temp_profiles) < 2:
        return None, None, None
    
    temp_array = np.array(temp_profiles)
    
    # Calculate mean and std, suppressing warnings
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        mean_temp = np.nanmean(temp_array, axis=0)
        std_temp = np.nanstd(temp_array, axis=0, ddof=1)
    
    # Check if we have valid data
    if np.all(np.isnan(mean_temp)):
        return None, None, None
    
    return depth_grid, mean_temp, std_temp


def create_animated_bundle_file():
    """Create animated bundle plot with mean profile option."""
    
    # Scan for populated year folders
    print("Scanning for redux folders...")
    available_years = []
    for year in range(2014, 2027):
        redux_dir = Path(f"~/redux{year}").expanduser()
        if redux_dir.exists():
            profile_count = len(list(redux_dir.glob("*.nc")))
            if profile_count > 0:
                print(f"  redux{year}: {profile_count} profiles")
                response = get_input_with_default(f"    Include {year}? [y/n] (default y):", "y").lower()
                if response == 'y':
                    available_years.append(year)
    
    if not available_years:
        print("No years selected")
        return
    
    print(f"\nSelected years: {available_years}")
    
    # Load all profile files from selected years
    profile_files = []
    for year in available_years:
        redux_dir = Path(f"~/redux{year}").expanduser()
        year_files = sorted(list(redux_dir.glob("*.nc")))
        profile_files.extend(year_files)
    
    print(f"Total profiles loaded: {len(profile_files)}")
    
    # Determine default date range from available files
    if profile_files:
        first_parts = profile_files[0].stem.split('_')
        last_parts = profile_files[-1].stem.split('_')
        
        first_year, first_doy = int(first_parts[4]), int(first_parts[5])
        last_year, last_doy = int(last_parts[4]), int(last_parts[5])
        
        default_start = datetime(first_year, 1, 1) + timedelta(days=first_doy - 1)
        default_end = datetime(last_year, 1, 1) + timedelta(days=last_doy - 1)
        
        default_start_str = default_start.strftime("%d-%b-%Y").upper()
        default_end_str = default_end.strftime("%d-%b-%Y").upper()
    else:
        default_start_str = "01-JAN-2018"
        default_end_str = "31-DEC-2018"
    
    # Get user inputs
    show_mean = get_input_with_default("Display mean profile? [y/n] (default y - shows mean):", "y").lower() == 'y'
    show_tmld = get_input_with_default("Include TMLD estimate in the visualization? Default is no. [y/n]", "n").lower() == 'y'
    n_profiles = int(get_input_with_default("How many profiles in the bundle? Default is 18 (two days)", "18"))
    delay = float(get_input_with_default("How many seconds delay between frames? (0.05 sec):", "0.05"))
    start_date = get_input_with_default(f"Start date (default {default_start_str}):", default_start_str)
    end_date = get_input_with_default(f"End date (default {default_end_str}):", default_end_str)
    
    # Parse dates
    start_dt = datetime.strptime(start_date, "%d-%b-%Y")
    end_dt = datetime.strptime(end_date, "%d-%b-%Y")
    
    tmld_df = load_tmld_data() if show_tmld else pd.DataFrame()
    
    # Filter files by date range
    filtered_files = []
    for file in profile_files:
        parts = file.stem.split('_')
        year = int(parts[4])
        doy = int(parts[5])
        file_date = datetime(year, 1, 1) + timedelta(days=doy - 1)
        if start_dt <= file_date <= end_dt:
            filtered_files.append(file)
    
    if len(filtered_files) < n_profiles:
        print(f"Only {len(filtered_files)} profiles found in date range")
        return
    
    print(f"Creating animation with {len(filtered_files)} profiles...")
    display_mode = "Mean Profile" if show_mean else "Bundle"
    print(f"Display mode: {display_mode}")
    
    # Set up the figure
    fig, ax = plt.subplots(figsize=(12, 8))
    total_frames = len(filtered_files) - n_profiles + 1
    
    def animate(frame):
        """Animation function."""
        ax.clear()
        ax.set_xlim(7, 20)
        ax.set_ylim(200, 0)
        ax.set_xlabel('Temperature (°C)', fontsize=12)
        ax.set_ylabel('Depth (m)', fontsize=12)
        ax.grid(True, alpha=0.3)
        
        start_idx = frame
        end_idx = min(start_idx + n_profiles, len(filtered_files))
        
        if start_idx >= len(filtered_files):
            return
        
        # Check for time gap
        has_time_gap = check_time_gap(filtered_files, start_idx, end_idx)
        
        if show_mean:
            # Calculate and plot mean profile
            depth_grid, mean_temp, std_temp = calculate_mean_profile(filtered_files, start_idx, end_idx)
            
            if depth_grid is not None:
                ax.plot(mean_temp, depth_grid, 'b-', linewidth=3, label='Mean')
                
                valid_mask = ~np.isnan(mean_temp) & ~np.isnan(std_temp)
                ax.plot(mean_temp[valid_mask] + std_temp[valid_mask], depth_grid[valid_mask], 
                       'b-', linewidth=1, alpha=0.5, label='+1 Std')
                ax.plot(mean_temp[valid_mask] - std_temp[valid_mask], depth_grid[valid_mask], 
                       'b-', linewidth=1, alpha=0.5, label='-1 Std')
                ax.legend(loc='lower right')
        else:
            # Plot bundle of profiles
            for i in range(start_idx, end_idx):
                try:
                    ds = xr.open_dataset(filtered_files[i])
                    temperature = ds['temperature'].values
                    depth = ds['depth'].values
                    
                    valid_mask = ~(np.isnan(temperature) | np.isnan(depth))
                    if np.any(valid_mask):
                        temp_clean = temperature[valid_mask]
                        depth_clean = depth[valid_mask]
                        
                        ax.plot(temp_clean, depth_clean, '-', linewidth=1, alpha=0.7)
                        
                        if show_tmld and not tmld_df.empty:
                            profile_idx = i + 1
                            tmld_row = tmld_df[tmld_df['profile_index'] == profile_idx]
                            if not tmld_row.empty and not np.isnan(tmld_row.iloc[0]['Estimated_TMLD']):
                                tmld_depth = tmld_row.iloc[0]['Estimated_TMLD']
                                tmld_temp = tmld_row.iloc[0]['temperature_at_TMLD']
                                if 7 <= tmld_temp <= 20:
                                    ax.plot(tmld_temp, tmld_depth, 'ro', markersize=4, alpha=0.8)
                    
                except Exception:
                    continue
        
        # Add Time Gap warning if needed
        if has_time_gap:
            ax.text(0.95, 0.05, 'Time Gap', transform=ax.transAxes,
                   fontsize=20, fontweight='bold', ha='right', va='bottom',
                   bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', linewidth=2))
        
        # Set title with date range
        if end_idx > start_idx:
            first_parts = filtered_files[start_idx].stem.split('_')
            last_parts = filtered_files[end_idx-1].stem.split('_')
            first_year, first_doy = int(first_parts[4]), int(first_parts[5])
            last_year, last_doy = int(last_parts[4]), int(last_parts[5])
            
            first_date = datetime(first_year, 1, 1) + timedelta(days=first_doy - 1)
            last_date = datetime(last_year, 1, 1) + timedelta(days=last_doy - 1)
            
            mode_str = " (Mean)" if show_mean else ""
            tmld_status = " (TMLD)" if show_tmld and not show_mean else ""
            title = f'Bundle Animation{mode_str}{tmld_status}: {first_date.strftime("%d-%b-%Y")} to {last_date.strftime("%d-%b-%Y")}'
            ax.set_title(title, fontsize=14)
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, frames=total_frames, 
                                 interval=delay*1000, repeat=True, blit=False)
    
    # Save animation to home directory
    output_file = Path("~/temp_bundle_animation.mp4").expanduser()
    print(f"Saving animation to {output_file}...")
    
    try:
        anim.save(str(output_file), writer='ffmpeg', fps=1/delay, dpi=100)
        
        if output_file.exists():
            file_size = output_file.stat().st_size / (1024*1024)
            print(f"Animation saved successfully!")
            print(f"File: {output_file}")
            print(f"Size: {file_size:.1f} MB")
            print(f"Frames: {total_frames}")
        else:
            print("Error: Output file was not created")
            
    except Exception as e:
        print(f"Error saving animation: {e}")
        print("Note: ffmpeg must be installed for MP4 output")
    
    plt.close(fig)

# Run the animation creation
create_animated_bundle_file()


Scanning for redux folders...
  redux2018: 1849 profiles


    Include 2018? [y/n] (default y):  


  redux2019: 2105 profiles


    Include 2019? [y/n] (default y):  



Selected years: [2018, 2019]
Total profiles loaded: 3954


Display mean profile? [y/n] (default y - shows mean):  
Include TMLD estimate in the visualization? Default is no. [y/n]  
How many profiles in the bundle? Default is 18 (two days)  
How many seconds delay between frames? (0.05 sec):  
Start date (default 01-JAN-2018):  
End date (default 27-SEP-2019):  


Creating animation with 3954 profiles...
Display mode: Mean Profile
Saving animation to /home/rob/temp_bundle_animation.mp4...
Animation saved successfully!
File: /home/rob/temp_bundle_animation.mp4
Size: 5.7 MB
Frames: 3937


In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import numpy as np

# Sensor configuration
sensor_map = {1: 'temperature', 2: 'salinity', 3: 'density', 4: 'dissolvedoxygen'}
sensor_defaults = {'temperature': (7.0, 20.0), 'salinity': (32.0, 34.0), 
                   'density': (1024.0, 1028.0), 'dissolvedoxygen': (50.0, 300.0)}
sensor_colors = {'temperature': 'red', 'salinity': 'blue', 
                 'density': 'black', 'dissolvedoxygen': 'cyan'}

# User inputs
years_input = input("Years to include (e.g., 2023,2024) [2023,2024]: ").strip() or "2023,2024"
years = [y.strip() for y in years_input.split(',')]

n_sensors = int(input("Number of sensors (1 or 2) [2]: ").strip() or "2")

sensors = []
ranges = []

defaults = [('temperature', 1), ('salinity', 2)]
for i in range(n_sensors):
    default_name, default_key = defaults[i] if i < len(defaults) else ('temperature', 1)
    key = int(input(f"Sensor {i+1} key (1=temperature, 2=salinity, 3=density, 4=dissolved oxygen) [{default_key}]: ").strip() or str(default_key))
    sensor = sensor_map[key]
    sensors.append(sensor)
    
    low_def, high_def = sensor_defaults[sensor]
    low = input(f"{sensor} low [{low_def}]: ").strip()
    high = input(f"{sensor} high [{high_def}]: ").strip()
    ranges.append((float(low) if low else low_def, float(high) if high else high_def))

# Get file lists (not loading data yet)
all_files = [[] for _ in range(n_sensors)]
for year in years:
    redux_folder = Path.home() / f'redux{year}'
    for i, sensor in enumerate(sensors):
        files = sorted(redux_folder.glob(f'RCA_sb_sp_{sensor}_*.nc'))
        all_files[i].extend(files)

print(f"\nFound {len(all_files[0])} profiles")

# Calculate chart ranges
if n_sensors == 1:
    a, b = ranges[0]
    chart_ranges = [(a, b)]
    print(f"\n{sensors[0]} has range {a} to {b}; chart has range {a} to {b}.")
else:
    a, b = ranges[0]
    c, d = ranges[1]
    chart_ranges = [(a, 2*b - a), (2*c - d, d)]
    print(f"\n{sensors[0]} has range {a} to {b}; chart first x-axis has range {a} to {2*b-a} to left-justify.")
    print(f"{sensors[1]} has range {c} to {d}; chart second x-axis has range {2*c-d} to {d} to right-justify.")

# Widgets
nProfiles_slider = widgets.IntSlider(min=0, max=180, value=1, description='nProfiles', continuous_update=False)
index0_slider = widgets.IntSlider(min=0, max=len(all_files[0])-1, value=0, description='index0', continuous_update=False)
btn_mm = widgets.Button(description='--')
btn_m = widgets.Button(description='-')
btn_p = widgets.Button(description='+')
btn_pp = widgets.Button(description='++')

# Plot type toggles
plot_toggles = []
for i, sensor in enumerate(sensors):
    toggle = widgets.ToggleButtons(options=['bundle', 'meanstd'], value='meanstd', 
                                   description=f'{sensor}:', button_style='')
    plot_toggles.append(toggle)

output = widgets.Output()

def update_plot(change=None):
    with output:
        output.clear_output(wait=True)
        
        fig, ax = plt.subplots(figsize=(14 if n_sensors == 2 else 10, 8))
        ax2 = ax.twiny() if n_sensors == 2 else None
        
        idx0 = index0_slider.value
        nProf = nProfiles_slider.value
        idx_end = min(idx0 + nProf, len(all_files[0]))
        
        # Time gap check - load only needed files
        if idx0 > 0:
            ds_prev = xr.open_dataset(all_files[0][idx0-1])
            ds_curr = xr.open_dataset(all_files[0][idx0])
            t_prev = ds_prev.time.values[-1]
            t_curr = ds_curr.time.values[0]
            gap_days = (t_curr - t_prev) / np.timedelta64(1, 'D')
            if gap_days > 2:
                ax.text(0.5, 0.98, 'Time Gap', transform=ax.transAxes, ha='center', 
                       va='top', fontsize=14, color='red', weight='bold')
        
        # Plot each sensor
        axes = [ax, ax2] if n_sensors == 2 else [ax]
        for i, sensor in enumerate(sensors):
            plot_type = plot_toggles[i].value
            
            if plot_type == 'bundle':
                for idx in range(idx0, idx_end):
                    ds = xr.open_dataset(all_files[i][idx])
                    axes[i].plot(ds[sensor].values, ds.depth.values, 
                               color=sensor_colors[sensor], alpha=0.3, linewidth=0.8)
            else:  # meanstd
                all_data = []
                common_depth = None
                for idx in range(idx0, idx_end):
                    ds = xr.open_dataset(all_files[i][idx])
                    depth = ds.depth.values
                    data = ds[sensor].values
                    if common_depth is None:
                        common_depth = depth
                    all_data.append(np.interp(common_depth, depth, data))
                
                if all_data:
                    data_array = np.array(all_data)
                    mean = np.nanmean(data_array, axis=0)
                    std = np.nanstd(data_array, axis=0)
                    axes[i].plot(mean, common_depth, color=sensor_colors[sensor], linewidth=2)
                    axes[i].fill_betweenx(common_depth, mean-std, mean+std, 
                                         color=sensor_colors[sensor], alpha=0.3)
        
        ax.set_ylim(200, 0)
        ax.set_ylabel('Depth (m)')
        ax.grid(True, alpha=0.3)
        ax.set_xlim(chart_ranges[0])
        ax.set_xlabel(sensors[0], color=sensor_colors[sensors[0]])
        ax.tick_params(axis='x', colors=sensor_colors[sensors[0]])
        
        if n_sensors == 2:
            ax2.set_xlim(chart_ranges[1])
            ax2.set_xlabel(sensors[1], color=sensor_colors[sensors[1]])
            ax2.tick_params(axis='x', colors=sensor_colors[sensors[1]])
            ax2.xaxis.set_label_position('top')
        
        plt.show()

def on_mm(b): index0_slider.value = max(0, index0_slider.value - nProfiles_slider.value//2)
def on_m(b): index0_slider.value = max(0, index0_slider.value - 1)
def on_p(b): index0_slider.value = min(index0_slider.max, index0_slider.value + 1)
def on_pp(b): index0_slider.value = min(index0_slider.max, index0_slider.value + nProfiles_slider.value//2)

btn_mm.on_click(on_mm)
btn_m.on_click(on_m)
btn_p.on_click(on_p)
btn_pp.on_click(on_pp)
index0_slider.observe(update_plot, 'value')
nProfiles_slider.observe(update_plot, 'value')
for toggle in plot_toggles:
    toggle.observe(update_plot, 'value')

display(widgets.HBox([btn_mm, btn_m, btn_p, btn_pp]))
for toggle in plot_toggles:
    display(toggle)
display(nProfiles_slider)
display(index0_slider)
display(output)
update_plot()



Years to include (e.g., 2023,2024) [2023,2024]:  2016
Number of sensors (1 or 2) [2]:  2
Sensor 1 key (1=temperature, 2=salinity, 3=density, 4=dissolved oxygen) [1]:  1
temperature low [7.0]:  
temperature high [20.0]:  
Sensor 2 key (1=temperature, 2=salinity, 3=density, 4=dissolved oxygen) [2]:  2
salinity low [32.0]:  31
salinity high [34.0]:  35



Found 2953 profiles

temperature has range 7.0 to 20.0; chart first x-axis has range 7.0 to 33.0 to left-justify.
salinity has range 31.0 to 35.0; chart second x-axis has range 27.0 to 35.0 to right-justify.


ToggleButtons(description='temperature:', index=1, options=('bundle', 'meanstd'), value='meanstd')

ToggleButtons(description='salinity:', index=1, options=('bundle', 'meanstd'), value='meanstd')

IntSlider(value=1, continuous_update=False, description='nProfiles', max=180)

IntSlider(value=0, continuous_update=False, description='index0', max=2952)

Output()